In [1]:
import cv2
import mediapipe as mp

# Predefined gestures
predefined_gestures = ["Right Point Thumb", "Left Point Thumb", "Thumb Up", "Thumb Down", "Pinch In", "Pinch Out"]

# Initialize the gesture-key map
gesture_key_map = {}

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

# Function to determine if the thumb is pointing right
def is_right_point_thumb(hand_landmarks, handedness):
    if handedness == "Right":
        return hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x > \
               hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x
    return False

# Function to determine if the thumb is pointing left
def is_left_point_thumb(hand_landmarks, handedness):
    if handedness == "Left":
        return hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x < \
               hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x
    return False

# Function to determine if thumb is up or down
def is_thumb_up_or_down(hand_landmarks):
    return "Up" if hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y < \
                   hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y else "Down"

# Function to determine if it's a pinch in or out
def is_pinch_in_or_out(hand_landmarks):
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    distance = ((thumb_tip.x - index_tip.x) ** 2 + (thumb_tip.y - index_tip.y) ** 2) ** 0.5
    return "Pinch In" if distance < 0.05 else "Pinch Out"  # Adjust threshold based on testing

# Process each frame
def process_frame(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            # Draw hand landmarks
            mp_draw.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Determine handedness
            hand_type = handedness.classification[0].label

            # Check each gesture
            if is_right_point_thumb(hand_landmarks, hand_type):
                print("Right Point Thumb Detected")
            elif is_left_point_thumb(hand_landmarks, hand_type):
                print("Left Point Thumb Detected")
            print(is_thumb_up_or_down(hand_landmarks))
            print(is_pinch_in_or_out(hand_landmarks))

    return image

# # Main function
# def main():
#     cap = cv2.VideoCapture(0)

#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             continue

#         frame = process_frame(frame)
#         cv2.imshow("Hand Gestures", frame)

#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()

# if __name__ == "__main__":
#     main()


import cv2
import mediapipe as mp
import pyautogui

def start_gesture_recognition():
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands()
    cap = cv2.VideoCapture(0)

    while cap.isOpened():
        success, image = cap.read()
        if not success:
            continue

        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        results = hands.process(image)
        if results.multi_hand_landmarks:
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                # Determine handedness
                hand_type = handedness.classification[0].label

                # Check each gesture and trigger corresponding key press
                if is_right_point_thumb(hand_landmarks, hand_type):
                    pyautogui.press(gesture_key_map.get("Right Point Thumb", ""))
                elif is_left_point_thumb(hand_landmarks, hand_type):
                    pyautogui.press(gesture_key_map.get("Left Point Thumb", ""))
                
                thumb_position = is_thumb_up_or_down(hand_landmarks)
                if thumb_position == "Up":
                    pyautogui.press(gesture_key_map.get("Thumb Up", ""))
                elif thumb_position == "Down":
                    pyautogui.press(gesture_key_map.get("Thumb Down", ""))
                
                pinch_gesture = is_pinch_in_or_out(hand_landmarks)
                if pinch_gesture == "Pinch In":
                    pyautogui.press(gesture_key_map.get("Pinch In", ""))
                elif pinch_gesture == "Pinch Out":
                    pyautogui.press(gesture_key_map.get("Pinch Out", ""))

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()



INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [1]:
import tkinter as tk
from tkinter import ttk, simpledialog
import threading
import pyautogui
import keyboard

def on_press(key):
    try:
        # Capture and format the key press
        key_name = key.char or key.name
        key_entry_var.set(key_name)
        # Stop listener
        return False
    except AttributeError:
        pass

def capture_key():
    with keyboard.Listener(on_press=on_press) as listener:
        listener.join()

def start_key_capture():
    threading.Thread(target=capture_key, daemon=True).start()

# ... rest of the GUI code ...

# Placeholder for gesture recognition logic
def start_gesture_recognition():
    print("Gesture recognition started.")
    # This will be replaced with the actual gesture recognition logic

def run_gesture_recognition():
    recognition_thread = threading.Thread(target=start_gesture_recognition)
    recognition_thread.start()

# Function to update gesture-key mapping
def update_mapping():
    gesture = gesture_var.get()
    key = key_entry.get()
    gesture_key_map[gesture] = key
    key_entry.delete(0, tk.END)
    update_label()

# Function to update the label showing current mappings
def update_label():
    mappings = '\n'.join([f'{g}: {k}' for g, k in gesture_key_map.items()])
    mapping_label.config(text=mappings)



# Set up the main window
root = tk.Tk()
root.title("Gesture to Key Mapping")

# Dropdown menu for gestures
gesture_var = tk.StringVar()
gesture_dropdown = ttk.Combobox(root, textvariable=gesture_var, values=predefined_gestures)
gesture_dropdown.pack()

# Entry for key
key_entry_var = tk.StringVar()
key_entry = tk.Entry(root, textvariable=key_entry_var)
key_entry.pack()

# Button to capture key
capture_key_button = tk.Button(root, text="Capture Key", command=start_key_capture)
capture_key_button.pack()

# Add a button to add new mappings
update_button = tk.Button(root, text="Add Gesture-Key Mapping", command=update_mapping)
update_button.pack()

# Add a button to start gesture recognition
start_button = tk.Button(root, text="Start Gesture Recognition", command=run_gesture_recognition)
start_button.pack()

# Add a label to show current mappings
mapping_label = tk.Label(root, text="No mappings yet")
mapping_label.pack()

# Start the GUI event loop
root.mainloop()


NameError: name 'predefined_gestures' is not defined

In [3]:
import cv2
import mediapipe as mp

# Predefined gestures
predefined_gestures = ["Right Point Thumb", "Left Point Thumb", "Thumb Up", "Thumb Down", "Pinch In", "Pinch Out"]

# Initialize the gesture-key map
gesture_key_map = {}

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

# Function to determine if the thumb is pointing right
def is_right_point_thumb(hand_landmarks, handedness):
    if handedness == "Right":
        return hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x > \
               hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x
    return False

# Function to determine if the thumb is pointing left
def is_left_point_thumb(hand_landmarks, handedness):
    if handedness == "Left":
        return hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x < \
               hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x
    return False

# Function to determine if thumb is up or down
def is_thumb_up_or_down(hand_landmarks):
    return "Up" if hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y < \
                   hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y else "Down"

# Function to determine if it's a pinch in or out
def is_pinch_in_or_out(hand_landmarks):
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    distance = ((thumb_tip.x - index_tip.x) ** 2 + (thumb_tip.y - index_tip.y) ** 2) ** 0.5
    return "Pinch In" if distance < 0.05 else "Pinch Out"  # Adjust threshold based on testing

# Process each frame
def process_frame(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            # Draw hand landmarks
            mp_draw.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Determine handedness
            hand_type = handedness.classification[0].label

            # Check each gesture
            if is_right_point_thumb(hand_landmarks, hand_type):
                print("Right Point Thumb Detected")
            elif is_left_point_thumb(hand_landmarks, hand_type):
                print("Left Point Thumb Detected")
            print(is_thumb_up_or_down(hand_landmarks))
            print(is_pinch_in_or_out(hand_landmarks))

    return image

# # Main function
# def main():
#     cap = cv2.VideoCapture(0)

#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             continue

#         frame = process_frame(frame)
#         cv2.imshow("Hand Gestures", frame)

#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()

# if __name__ == "__main__":
#     main()


import cv2
import mediapipe as mp
import pyautogui

def start_gesture_recognition():
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands()
    cap = cv2.VideoCapture(0)

    while cap.isOpened():
        success, image = cap.read()
        if not success:
            continue

        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        results = hands.process(image)
        if results.multi_hand_landmarks:
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                # Determine handedness
                hand_type = handedness.classification[0].label

                # Check each gesture and trigger corresponding key press
                if is_right_point_thumb(hand_landmarks, hand_type):
                    pyautogui.press(gesture_key_map.get("Right Point Thumb", ""))
                elif is_left_point_thumb(hand_landmarks, hand_type):
                    pyautogui.press(gesture_key_map.get("Left Point Thumb", ""))
                
                thumb_position = is_thumb_up_or_down(hand_landmarks)
                if thumb_position == "Up":
                    pyautogui.press(gesture_key_map.get("Thumb Up", ""))
                elif thumb_position == "Down":
                    pyautogui.press(gesture_key_map.get("Thumb Down", ""))
                
                pinch_gesture = is_pinch_in_or_out(hand_landmarks)
                if pinch_gesture == "Pinch In":
                    pyautogui.press(gesture_key_map.get("Pinch In", ""))
                elif pinch_gesture == "Pinch Out":
                    pyautogui.press(gesture_key_map.get("Pinch Out", ""))

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()



import tkinter as tk
from tkinter import ttk, simpledialog
import threading
import pyautogui
from pynput import keyboard

def on_press(key):
    try:
        # Capture and format the key press
        key_name = key.char or key.name
        key_entry_var.set(key_name)
        # Stop listener
        return False
    except AttributeError:
        pass

def capture_key():
    with keyboard.Listener(on_press=on_press) as listener:
        listener.join()

def start_key_capture():
    threading.Thread(target=capture_key, daemon=True).start()

# ... rest of the GUI code ...

# Placeholder for gesture recognition logic
# def start_gesture_recognition():
#     print("Gesture recognition started.")
    # This will be replaced with the actual gesture recognition logic

def run_gesture_recognition():
    recognition_thread = threading.Thread(target=start_gesture_recognition)
    recognition_thread.start()

# Function to update gesture-key mapping
def update_mapping():
    gesture = gesture_var.get()
    key = key_entry.get()
    gesture_key_map[gesture] = key
    key_entry.delete(0, tk.END)
    update_label()

# Function to update the label showing current mappings
def update_label():
    mappings = '\n'.join([f'{g}: {k}' for g, k in gesture_key_map.items()])
    mapping_label.config(text=mappings)



# Set up the main window
root = tk.Tk()
root.title("Gesture to Key Mapping")

# Dropdown menu for gestures
gesture_var = tk.StringVar()
gesture_dropdown = ttk.Combobox(root, textvariable=gesture_var, values=predefined_gestures)
gesture_dropdown.pack()

# Entry for key
key_entry_var = tk.StringVar()
key_entry = tk.Entry(root, textvariable=key_entry_var)
key_entry.pack()

# Button to capture key
capture_key_button = tk.Button(root, text="Capture Key", command=start_key_capture)
capture_key_button.pack()

# Add a button to add new mappings
update_button = tk.Button(root, text="Add Gesture-Key Mapping", command=update_mapping)
update_button.pack()

# Add a button to start gesture recognition
start_button = tk.Button(root, text="Start Gesture Recognition", command=run_gesture_recognition)
start_button.pack()

# Add a label to show current mappings
mapping_label = tk.Label(root, text="No mappings yet")
mapping_label.pack()

# Start the GUI event loop
root.mainloop()


KeyboardInterrupt: 

: 